In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import datetime
now = datetime.datetime.now()

In [174]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
year = 2010 # Year we're currently testing.
html = requests.get('https://en.wikipedia.org/wiki/List_of_Billboard_Hot_100_top-ten_singles_in_' + str(year),cookies={ 'sessionid': '123..'},headers=headers).text
soup = BeautifulSoup(html, 'lxml')
print(soup.prettify()[0:1000])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Billboard Hot 100 top-ten singles in 2010 - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XgWZHQpAAEYAAIfWb9YAAACJ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Billboard_Hot_100_top-ten_singles_in_2010","wgTitle":"List of Billboard Hot 100 top-ten singles in 2010","wgCurRevisionId":899254653,"wgRevisionId":899254653,"wgArticleId":25552100,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":

In [175]:
my_table = soup.find('table',{'class':'wikitable sortable'})
type(my_table)

bs4.element.Tag

In [176]:
swiftcount = 0 # Keeps track of where repeat information from cells with rowspans greater than 1 should be inserted.
drakecontingency = 0 # Used to keep track of how many times repeat information should be inserted.
swiftcount2 = 0 # Used in the rare case that two rowspan situations overlap.
drakecontingency2 = 0

# The following code mines the first table found on the Wikipedia page and extracts all table elements other than the annotations column.

In [177]:
Titles = []
try:
    tds = my_table.findAll('td')
except AttributeError:
    my_table = soup.find('table',{'class':'wikitable'})
    tds = my_table.findAll('td')

for td in tds:
        try:
            if td.attrs['rowspan']:
                if drakecontingency == 0:
                    taylorswift = td.get_text()[:-1]
                    swiftcount = 8
                    drakecontingency = int(td.attrs['rowspan']) - 1
                else:
                    taylorswift2 = td.get_text()[:-1]
                    swiftcount2 = 7
                    drakecontingency2 = int(td.attrs['rowspan']) - 1
        except KeyError:
            pass
        if td.get_text()[0] != '[' and td.get_text()[0] and td.get_text()[0].strip(): # Drop annotations column.
            if swiftcount > 0:
                swiftcount = swiftcount-1
            if swiftcount == 1:
                drakecontingency = drakecontingency-1
                print(taylorswift + ' appended!')
                Titles.append(taylorswift)
                if drakecontingency == 0:
                    taylorswift = ''
                    swiftcount = 0
                else:
                    if drakecontingency2 == 0: # If there's an extra element being inserted before the next time this runs, insert it one sooner next time.
                        swiftcount = 6
                    else:
                        swiftcount = 5
            if swiftcount2 > 0:
                swiftcount2 = swiftcount2-1
            if swiftcount2 == 1:
                drakecontingency2 = drakecontingency2-1
                print(taylorswift2 + ' appended!')
                Titles.append(taylorswift2)
                if drakecontingency2 == 0:
                    taylorswift2 = ''
                    swiftcount2 = 0
                else:
                    if drakecontingency == 0:
                        swiftcount2 = 6
                    else:
                        swiftcount2 = 5
            Titles.append(td.get_text()[:-1]) # Drop the newline character present at the end of every single entry.

November 28 appended!
January 23 appended!
January 23 appended!
January 30 appended!
February 6 appended!
February 27 appended!
March 27 appended!
May 1 appended!
May 22 appended!
May 29 appended!
June 19 appended!
July 10 appended!
August 14 appended!
September 4 appended!
September 4 appended!
October 2 appended!
Taylor Swift appended!
October 30 appended!
November 20 appended!


In [178]:
print(Titles)

['November 21', '"Replay"', 'Iyaz', '2', 'January 9', '14', 'November 28', '"Need You Now" (#2)', 'Lady Antebellum', '2', 'March 20', '16', 'November 28', '"Tik Tok" (#1)', 'Kesha', '1', 'January 2', '20', 'December 5', '"Sexy Bitch"', 'David Guetta featuring Akon', '5', 'February 13', '13', 'January 9', '"Do You Remember"', 'Jay Sean featuring Sean Paul and Lil Jon', '10', 'January 9', '1', 'January 23', '"Blah Blah Blah" ◁', 'Kesha featuring 3OH!3', '7', 'January 23', '1', 'January 23', '"BedRock"', 'Young Money featuring Lloyd', '2', 'March 13', '13', 'January 23', '"Hard"', 'Rihanna featuring Young Jeezy', '8', 'January 30', '4', 'January 30', '"Hey, Soul Sister" (#3) [A]', 'Train', '3', 'April 10', '19', 'January 30', '"How Low"', 'Ludacris', '6', 'February 13', '7', 'February 6', '"Baby" ◁', 'Justin Bieber featuring Ludacris', '5', 'February 6', '5', 'February 6', '"Today Was a Fairytale" ◁', 'Taylor Swift', '2', 'February 6', '1', 'February 13', '"Imma Be"', 'The Black Eyed Peas

In [179]:
nptitles = np.asarray(Titles).copy()

In [180]:
len(nptitles)

354

In [181]:
numsongs = int(np.prod(nptitles.shape)/6) # Find out how many songs are in this list.
reshaped = nptitles.reshape(numsongs,6) # Reshape the array based on the number of songs and the number of columns.

In [182]:
songdata = pd.DataFrame(reshaped)
songdata.columns = ['Top ten entry date','Title','Artist','Peak','Peak date','Number of weeks in top ten']
songdata['Year'] = str(year)
songdata['Title'] = [i[1:i.rfind('\"')] for i in songdata['Title']] # Drops quotation marks and annotations in Title.
for j in songdata['Top ten entry date']:
    if '[' in j:
        j = j[:j.rfind('[')-1]

if year == now.year:
    cleanweeks = []
    for j in songdata['Number of weeks in top ten']: # This code searches the number of weeks in top ten column for asterisks and eliminates them. This is only a 
        try:                                         # problem in the current year.
            cleanweeks.append(int(j[:j.rindex('*')]))
        except ValueError:
            cleanweeks.append(int(j))

    songdata['Number of weeks in top ten'] = cleanweeks

In [183]:
songdata

,Top ten entry date,Title,Artist,Peak,Peak date,Number of weeks in top ten,Year
0,November 21,Replay,Iyaz,2,January 9,14,2010
1,November 28,Need You Now,Lady Antebellum,2,March 20,16,2010
2,November 28,Tik Tok,Kesha,1,January 2,20,2010
3,December 5,Sexy Bitch,David Guetta featuring Akon,5,February 13,13,2010
4,January 9,Do You Remember,Jay Sean featuring Sean Paul and Lil Jon,10,January 9,1,2010
5,January 23,Blah Blah Blah,Kesha featuring 3OH!3,7,January 23,1,2010
6,January 23,BedRock,Young Money featuring Lloyd,2,March 13,13,2010
7,January 23,Hard,Rihanna featuring Young Jeezy,8,January 30,4,2010
8,January 30,"Hey, Soul Sister",Train,3,April 10,19,2010
9,January 30,How Low,Ludacris,6,February 13,7,2010


In [173]:
songdata.to_csv(path_or_buf='data/' + str(year) + 'top10songs.csv',index=False) # Export our sample data table.